In [1]:
%load_ext autoreload

In [4]:
import sys
loder_path = "/home/chopprin/kaggle/attention_branch_network/attention_branch_network-keras/utils"
sys.path.append("loder_path")

In [13]:
%aimport utils.loader
from utils.loader import DataLoader
from solver import Solver
import numpy as np

In [6]:
# <mainの中>
# 各パラメータ設定
import argparse
parser = argparse.ArgumentParser()
parser.add_argument("--mode", type=str, default="train")
parser.add_argument("--batch_size", type=int, default=16)
parser.add_argument("--pad_size", type=int, default=2)
parser.add_argument("--crop_size", type=int, default=32)
parser.add_argument("--mixed_training", type=bool, default=False)
parser.add_argument("--n_epochs", type=int, default=100)
parser.add_argument("--lr", type=float, default=1e-3)
parser.add_argument("--save_dir", type=str, default="./logs")
args = parser.parse_args(args=[])
#main(args)

In [7]:
# Dataloderクラスを使う
train_loader = DataLoader(mode = "train", batch_size=args.batch_size,\
                         pad_size=args.pad_size, crop_size=args.crop_size)
valid_loader = DataLoader(mode = "valid", batch_size=args.batch_size**2)

x_train.shape(60000, 28, 28, 1)
x_test.shape(10000, 28, 28, 1)
y_train.shape(60000, 10)
y_test.shape(10000, 10)
x_train.shape(60000, 28, 28, 1)
x_test.shape(10000, 28, 28, 1)
y_train.shape(60000, 10)
y_test.shape(10000, 10)


In [8]:
# Sloverクラスを使う
solver_instance = Solver(args= args, data_loader=train_loader, valid_loader=valid_loader)

model input : Tensor("input_image:0", shape=(None, 28, 28, 1), dtype=float32)
model output : [<tf.Tensor 'attention_branch_output/Identity:0' shape=(None, 10) dtype=float32>, <tf.Tensor 'perception_branch_output/Identity:0' shape=(None, 10) dtype=float32>]


In [10]:
model = solver_instance.model

In [20]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_image (InputLayer)        [(None, 28, 28, 1)]  0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 28, 28, 256)  2304        input_image[0][0]                
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 28, 28, 256)  1024        conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 28, 28, 256)  0           batch_normalization[0][0]        
______________________________________________________________________________________________

In [16]:
model.fit(x = train_loader.x_train, y = [train_loader.y_train, train_loader.y_train],
          batch_size=train_loader.batch_size, epochs=1,
          verbose=1,validation_split = 0.0)

Train on 48000 samples
   16/48000 [..............................] - ETA: 18:07

ValueError: Shape mismatch: The shape of labels (received (160,)) should equal the shape of logits except for the last dimension (received (16, 10)).

In [38]:
# Attention Branch
#Attention Branch Network

# 分岐型ネットワークでの定義
import tensorflow as tf
from keras.layers import Input, Dense, Flatten, Conv2D, Activation,\
                        MaxPool2D, GlobalAveragePooling2D, BatchNormalization
from keras.layers.merge import add, multiply
from keras import backend as K

original_im = Input(shape=(28,28,1))
x1 = Conv2D(filters = 32, kernel_size = (5,5), padding = 'Same',
           activation='relu', input_shape = (28,28,1))(original_im)
x1 = Conv2D(filters = 64, kernel_size = (5,5,), padding = "Same",
           activation="relu")(x1)
feature_map = MaxPool2D(pool_size = (2,2), strides = (2,2))(x1)

# Attention Branch
x2 = Conv2D(filters = 64, kernel_size = (5,5,), padding = "Same",
           activation="relu")(feature_map) # Attention Branchの具体的なClassifierの構造はここを変化させる．
#x2 = BatchNormalization(axis = 1)(x2) # ★ axis = 1であってるか確認
x2 = Conv2D(filters = 64, kernel_size = (1,1,), padding = "Same",
           activation="relu")(x2)
x2_1 = Conv2D(filters = 64, kernel_size = (1,1,), padding = "Same",
           activation="relu")(x2)

# Attention Branch 分岐１
x2_1 = Conv2D(filters = 64, kernel_size = (1,1,), padding = "Same")(x2_1)
x2_1 = GlobalAveragePooling2D(data_format = "channels_last")(x2_1) 
x2_1 = Dense(10, activation = "softmax")(x2_1)
output1 = Activation("softmax")(x2_1)

# Attention Branch 分岐2
x2 = Conv2D(filters = 1, kernel_size = (1,1,), padding = "Same")(x2)
x2 = BatchNormalization()(x2)
attention_map = Activation("sigmoid")(x2)

# Perception Branch
# ★ 足し算のやり方調べる．
#one = K.variable(np.ones((14,14,64)))
#kari = add([feature_map, one], axis = 2)
x4 = multiply([feature_map, attention_map])
# ★ PerceptionBranchの具体的なclassifierの構造はここを変化させる．
x4 = Conv2D(filters = 64, kernel_size = (3,3,), padding = "Same",
           activation="relu")(x4)
x4 = MaxPool2D(pool_size = (2,2), strides = (2,2))(x4)
x4 = Flatten(data_format=None)(x4)
x4 = Dense(512, activation = "relu")(x4)
output2 = Dense(10, activation = "softmax")(x4)

ab_model = keras.models.Model(inputs = original_im, outputs = [output1, output2])

optimizer = RMSprop(lr=0.001, rho=0.9, epsilon=1e-08, decay=0.0)
ab_model.compile(optimizer=optimizer, 
              loss=["categorical_crossentropy", "categorical_crossentropy"], 
              metrics=['accuracy'])

epochs = 1 # Turn epochs to 30 to get 0.9967 accuracy
batch_size = 86

# Y_trainはリスト2つに
ab_model_history = ab_model.fit(x = train_loader.x_train[0:1000], y = [train_loader.y_train[0:1000], train_loader.y_train[0:1000]], validation_split = 0.0, \
                 epochs=epochs, batch_size = batch_size)

Epoch 1/1
1000/1000 [==============================] - 7s 7ms/step - loss: 11.0055 - activation_3_loss: 2.3533 - dense_8_loss: 8.4224 - activation_3_accuracy: 0.0990 - dense_8_accuracy: 0.3890


In [42]:
ab_model.input

<tf.Tensor 'input_5:0' shape=(None, 28, 28, 1) dtype=float32>

In [43]:
ab_model.output

[<tf.Tensor 'activation_3_1/Softmax:0' shape=(None, 10) dtype=float32>,
 <tf.Tensor 'dense_8/Softmax:0' shape=(None, 10) dtype=float32>]

In [ ]:
model.fit( self.model.fit_generator(
            generator=self.data_loader,
            validation_data=self.valid_loader,
            epochs=self.n_epochs,
            steps_per_epoch=len(self.data_loader),
            validation_steps=len(self.valid_loader),
            verbose=1,
            workers=10,
            max_queue_size=30,
            callbacks=callbacks)

In [13]:
solver_instance.train()

Instructions for updating:
Please use Model.fit, which supports generators.


ValueError: operands could not be broadcast together with remapped shapes [original->remapped]: (3,2) and requested shape (2,2)

In [48]:
# modelの形をゲットする．
from utils.modules import get_model

num_class = 10

model = get_model((28, 28, 1), num_class, out_ch = 256, n = 1)

データローダーの自作